In [ ]:
import pandas as pd
from pandas import DataFrame
from functools import partial

In [ ]:
%pwd
%cd "~/Documents/GitHub/smart-tab-grouping"


In [ ]:
from rouge_score import rouge_scorer

In [ ]:
multitab_tests = pd.read_csv("data/individual_tests/private/all_users2.csv")
single_tab_tests = pd.read_csv("data/individual_tests/single_tab_validation.csv")
single_tab_tests.keywords = ""

In [ ]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, pipeline
import numpy as np

In [ ]:
embedder = pipeline("feature-extraction", model="sentence-transformers/all-MiniLM-L6-v2", device=-1)


In [ ]:
def cos_sim(s1, s2):
    embeddings = [np.mean(embedder(s)[0], axis=0) for s in [s1, s2]]
    similarity = cosine_similarity(embeddings[0].reshape(1,-1), embeddings[1].reshape(1,-1)).squeeze()
    return similarity



In [ ]:
cos_sim("Dogs", "Apple")

In [ ]:
def compute_scores(row, pred_key=None):
    scores = scorer.score(row['label'], row[pred_key])
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure,
        'pred_len': len(row[pred_key]),
        'label_len': len(row['label']),
        'cos_sim': cos_sim(row['label'], row[pred_key])
    }


In [ ]:
def get_avg_scores(input_df: DataFrame, compare_column: str):
    rouge_scores_df = input_df.apply(partial(compute_scores, pred_key=compare_column) , axis=1, result_type='expand')
    average_scores = rouge_scores_df.mean().to_dict()
    return average_scores


In [ ]:
import sys
sys.path.append("/Users/Rrando/Documents/GitHub/smart-tab-grouping/src")
from util.tab_titles import T5TopicGenerator

In [ ]:
def compute_topic_keywords(row, legacy=False, prob_limit=None):
    return topic_gen.get_topic_with_keywords({"documents": row["three_titles"].split('\n'), "keywords": row["keywords"].split(',')}, legacy=legacy, prob_limit=prob_limit)

In [ ]:
topic_gen = T5TopicGenerator()

In [ ]:
def compute_topic_keywords_single(row, legacy=False, prob_limit=None):
    return topic_gen.get_topic_with_keywords({"documents": [row["title"]], "keywords": row["keywords"].split(',')}, legacy=legacy, 
                                             prob_limit=prob_limit)

In [ ]:
def compute_topic(row):
    return topic_gen.get_topic({"documents": row["three_titles"].split('\n')})

In [ ]:

multitab_tests["recomputed_titles_keywords"] = multitab_tests.apply(lambda row: compute_topic_keywords(row), axis=1)
#multitab_tests["recomputed_title_no_keywords"] = multitab_tests.apply(lambda row: compute_topic(row), axis=1)
# Look at OpenAI can do for generating topics from a set of tabs
#llm_topic_gen_no_keywords = OpenAITopicGenerator(support_keywords=False)
#multitab_tests["openai_keywords"] = multitab_tests.apply(lambda row: llm_topic_gen_keywords.get_topic({"documents": row.three_titles.split("\n"), "keywords": row.keywords.split(",")}), axis=1)
#print_rouge_scores(multitab_tests, "openai_keywords")
# Compare with fine tuned model
#topic_gen = T5TopicGenerator(model_name="./models/gentle-pyramid-114")
#multitab_tests["recomputed_title_keywords_pyramid"] = multitab_tests.apply(lambda row: compute_topic_keywords(row, legacy=False), axis=1)
#print_rouge_scores(multitab_tests, "recomputed_title_keywords_pyramid")


In [ ]:
#           {"name": "cool-yogurt-98", "legacy_data_format": False},
#          {"name": "dainty-blaze-127", "legacy_data_format": False},
#          {"name": "major-cloud-188", "legacy_data_format": False},


models = [
           {"name": "cool-yogurt-98", "legacy_data_format": False},
          {"name": "dainty-blaze-127", "legacy_data_format": False},
        {"name": "dainty-river-189","legacy_data_format": False},
        {"name": "gallant-sunset-190","legacy_data_format": False},
        {"name": "upbeat-eon-195", "legacy_data_format": False},
        {"name": "devoted-puddle-246", "legacy_data_format": False},
        {"name": "genial-tree-283", "legacy_data_format": False},

         ]

# pious-butterfly-170 is broken


In [ ]:
single_tab_tests["keywords"] = pd.Series(dtype=str)
single_tab_tests = single_tab_tests.fillna("")

In [ ]:
single_tab_score = []
multi_tab_score = []

for model_info in models:
    name = model_info["name"]
    topic_gen = T5TopicGenerator(model_name=f"./models/{name}")
    col = f"recomputed_title_keywords_{name}"
    multitab_tests[col] = multitab_tests.apply(lambda row: compute_topic_keywords(row, legacy=model_info["legacy_data_format"]), axis=1)
    print(f"{name} - MultiTab Tests")
    score = get_avg_scores(multitab_tests, col)
    score["model"] = name
    multi_tab_score.append(score)
    
    single_tab_tests[col] = single_tab_tests.apply(lambda row: compute_topic_keywords_single(row, legacy=model_info["legacy_data_format"]), axis=1)
    print(f"{name} - Single Tab Tests")
    score = get_avg_scores(single_tab_tests, col)
    score["model"] = name
    single_tab_score.append(score)

    
    

In [ ]:
single_tab_df = pd.DataFrame(single_tab_score)
multi_tab_df = pd.DataFrame(multi_tab_score)

In [ ]:
multi_tab_df

In [ ]:
single_tab_df